In [183]:
#load file
def txt_strtonum_feed(filename):
    data = []
    data_str = ''
    with open(filename, 'r', encoding='UTF-8') as f:#with auto call close()
        line = f.readline()
        while line:
            #read_data = line.split('\n')
            #data_str +=read_data
            data_str +=line
            #data.append(read_data[0])
            line = f.readline()
        return data_str 

In [184]:
import xml.sax
from xml.dom.minidom import parse
import xml.dom.minidom
import re
from stanfordcorenlp import StanfordCoreNLP
nlp=StanfordCoreNLP('G:\\stanford-corenlp-full-2018-10-05',lang='en')

In [185]:
filename = "data/11532192.txt"
ann_filename = "data/11532192.xml"

In [186]:
    txt = txt_strtonum_feed(filename)
    DOMTree = xml.dom.minidom.parse(ann_filename)
    collection = DOMTree.documentElement
    annotations = collection.getElementsByTagName("annotation")
    data_list = []
    for annotation in annotations:
        mention = annotation.getElementsByTagName('class')[0].getAttribute("id")

        for span in annotation.getElementsByTagName('span'):
            end = int(span.getAttribute("end")  )   
            start =int(span.getAttribute("start")) 
            data = span.childNodes[0].data
            data_list.append((start,end,mention,data))
    data_list.sort()
    
    #处理注释数据
    handle_data = []
    for i in data_list:
        if handle_data:
            if handle_data[-1][0] == i[0] and handle_data[-1][1] == i[1]:
                continue
            elif handle_data[-1][0] == i[0] and handle_data[-1][1] > i[1]:
                continue
            elif handle_data[-1][0] == i[0] and handle_data[-1][1] < i[1]:
                temp = handle_data[-1]
                handle_data.remove(temp)

            elif handle_data[-1][0] < i[0] and handle_data[-1][1] == i[1]:
                continue
            elif handle_data[-1][0] > i[0] and handle_data[-1][1] == i[1]:
                temp = handle_data[-1]
                handle_data.remove(temp)
            elif handle_data[-1][1] > i[0]:
                continue

        handle_data.append(i)
    data_list = handle_data

In [187]:
    spanStart_list=[]
    spanEnd_list=[]
    spannedText_list=[]
    pr_list=[]
    for i in data_list:
        spanStart_list.append(i[0])
        spanEnd_list.append(i[1])
        spannedText_list.append(i[3])
        pr_list.append(i[2]) 

In [188]:

"""try to custom the rules"""
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
punkt_param = PunktParameters()
abbreviation = ['i.e']
punkt_param.abbrev_types = set(abbreviation)
sent_tokenizer = PunktSentenceTokenizer(punkt_param)

In [15]:

"""try to custom the rules"""
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
punkt_param = PunktParameters()
abbreviation = ['i.e']
punkt_param.abbrev_types = set(abbreviation)
sent_tokenizer = PunktSentenceTokenizer(punkt_param)

In [189]:
    sentence_list =[]
    data = re.split("\. |\n\n",txt)


In [190]:

    sentence_list = []
    j=0
    number_=0
    last_number=0
    sentence_list=[]
    new_star_list = []
    new_end_list = []
    for i in range(len(spanStart_list)):
        while number_ <= int(spanStart_list[i]):
            sentence_length = 0
            last_number = number_
            for k in range(len(data[j])):
                sentence_length+=len(str(data[j][k]))
                number_+= len(str(data[j][k]))
            number_+= 2
            j+=1;

        sentence_list.append(data[j-1])
        new_star_list.append(int(spanStart_list[i])-last_number)
        new_end_list.append(int(spanEnd_list[i])-last_number)

In [191]:
    store_list = []
    store_index = 0
    annote_list=[]
    sub_annote_list=[]
    for i in range(len(sentence_list)):
        if i == 0:
            store_list.append(sentence_list[i])
            sub_annote_list.append((spannedText_list[i],pr_list[i],new_star_list[i],new_end_list[i]))
            annote_list.append(sub_annote_list)

        elif sentence_list[i]==sentence_list[i-1]:

            sub_annote_list.append((spannedText_list[i],pr_list[i],new_star_list[i],new_end_list[i]))

        else:
            sub_annote_list=[]
            store_index+=1
            store_list.append(sentence_list[i])
            sub_annote_list.append((spannedText_list[i],pr_list[i],new_star_list[i],new_end_list[i]))
            annote_list.append(sub_annote_list)

In [192]:
annote_list[0]

[('ocular', 'UBERON:0000970', 5, 11),
 ('genetically', 'SO:0000704', 24, 35),
 ('mice', 'NCBITaxon:10088', 45, 49)]

In [193]:
store_list[-1][39:43]

'mice'

In [194]:
error_list = []
right_ok=[]
for i in range(len(store_list)) :
    for j in annote_list[i]:
        if store_list[i][j[2] : j[3]] != j[0]:
            error_list.append((i,j))
        else:
            right_ok.append((i,j))

In [195]:
error_list

[]

In [119]:
len(store_list[0])

78

In [129]:
store_list[0][77]

'y'

In [130]:
store_list[0]

'Intraocular pressure in genetically distinct mice: an update and strain survey'

In [196]:


    doc_out_put = []
    error_list = []
    mark_list = ['(']
    for i in range(len(store_list)):
        sub_output = []
        doc = nlp.pos_tag(store_list[i])
        index_start = 0
        index_end = 0
        while store_list[i][index_start]==" ":
            index_start += 1
            index_end += 1
        
        for j in range(len(doc)):
            index_end = index_end + len(doc[j][0])
            sub_output.append((doc[j][0],doc[j][1],index_start,index_end))
            if index_end < len(store_list[i])-1:
                if store_list[i][index_end] == " ":
                    index_end += 1
            index_start = index_end
        doc_out_put.append(sub_output)
        
        
        

In [198]:
store_list

['Intraocular pressure in genetically distinct mice: an update and strain survey',
 'Little is known about genetic factors affecting intraocular pressure (IOP) in mice and other mammals',
 'The purpose of this study was to determine the IOPs of genetically distinct mouse strains, assess the effects of factors such as age, sex and time of day on IOP in specific strain backgrounds, and to assess the effects of specific candidate gene mutations on IOP.',
 'Based on over 30 studied mouse strains, average IOP ranges from approximately 10 to 20 mmHg',
 'Gender does not typically affect IOP and aging results in an IOP decrease in some strains',
 'Most tested strains exhibit a diurnal rhythm with IOP being the highest during the dark period of the day',
 'Homozygosity for a null allele of the carbonic anhydrase II gene (Car2n) does not alter IOP while homozygosity for a mutation in the leptin receptor gene (Leprdb) that causes obesity and diabetes results in increased IOP',
 'Albino C57BL/6J m

In [197]:
doc_out_put

[[('Intraocular', 'JJ', 0, 11),
  ('pressure', 'NN', 12, 20),
  ('in', 'IN', 21, 23),
  ('genetically', 'RB', 24, 35),
  ('distinct', 'JJ', 36, 44),
  ('mice', 'NNS', 45, 49),
  (':', ':', 49, 50),
  ('an', 'DT', 51, 53),
  ('update', 'VB', 54, 60),
  ('and', 'CC', 61, 64),
  ('strain', 'VB', 65, 71),
  ('survey', 'NN', 72, 78)],
 [('Little', 'JJ', 0, 6),
  ('is', 'VBZ', 7, 9),
  ('known', 'VBN', 10, 15),
  ('about', 'IN', 16, 21),
  ('genetic', 'JJ', 22, 29),
  ('factors', 'NNS', 30, 37),
  ('affecting', 'VBG', 38, 47),
  ('intraocular', 'JJ', 48, 59),
  ('pressure', 'NN', 60, 68),
  ('(', '-LRB-', 69, 70),
  ('IOP', 'NN', 70, 73),
  (')', '-RRB-', 73, 74),
  ('in', 'IN', 75, 77),
  ('mice', 'NNS', 78, 82),
  ('and', 'CC', 83, 86),
  ('other', 'JJ', 87, 92),
  ('mammals', 'NNS', 93, 100)],
 [('The', 'DT', 0, 3),
  ('purpose', 'NN', 4, 11),
  ('of', 'IN', 12, 14),
  ('this', 'DT', 15, 19),
  ('study', 'NN', 20, 25),
  ('was', 'VBD', 26, 29),
  ('to', 'TO', 30, 32),
  ('determine', 'VB'

In [199]:
error_list = []
right_ok=[]
for i in range(len(store_list)) :
    for j in doc_out_put[i]:
        if store_list[i][j[2] : j[3]] != j[0]:
            error_list.append((i,j))
        else:
            right_ok.append((i,j))

In [200]:
error_list

[]

In [179]:
annote_list[-5]

[('Exon', 'SO:0000147', 0, 4)]

In [182]:
store_list[-6]

"Exon 2: 5'-tgaagccatactttaccaaccat-3' and 5'-caaaagggagaagtctaacttc-3'"

In [181]:
annote_list[-6]

[('Exon', 'SO:0000147', 0, 4)]

In [91]:
         
            
    #处理注释文件，包括文献文本  并且标注数据   
    record_error = []
    out_put = []
    for i in range(len(store_list)):
        doc = nlp.pos_tag(store_list[i])
        sub_error=[]
        k=0
        sub_output = []
        flag=False
        flag_index=0
        is_jump = False
        
        index_start = 0
        index_end = 0
        
            
        for j in range(len(doc)):
            
            index_end = index_end + len(doc[j][0])
            
            if is_jump:
                is_jump = False
                
                index_end = index_end + 1
                index_start = index_end
                continue
            
            
            split_annote = annote_list[i][k][0].split(" ")
            split1 = doc[j][0].split("/") 
            split2 = doc[j][0].split("-")
            split3 = "#$%^&"
            if len(split2) > 1:
                split3 = split2[1]
            split4 = annote_list[i][k][0].replace("-","")
            #find_inner = False
            find_inner = (annote_list[i][k][0] in doc[j][0]) and (len(annote_list[i][k][0]) > 4)
            #find_inner1 = False
            find_inner1 = (doc[j][0] in split_annote[0]) and (len(doc[j][0]) > 4)
            
            find_inner2=False
            if flag_index==0:
                find_inner2 = split_annote[0] in doc[j][0] and (len(split_annote[0]) > 4)
            
            combine_string = ""
            
            if j < len(doc)-1:
                combine_string = doc[j][0]+doc[j+1][0]
            find_combin = (combine_string == split_annote[0] )
            
#             if i == 143:
#                 print(k)
#                 print("flag_index",flag_index)
#                 print("in sentence: ",doc[j])
#                 print("need find: ",split_annote[flag_index])
#                 print("find_inner2: ",find_inner2)
                
                
            #如果注释在文本中的话
            if (split4==doc[j][0]) or find_inner or (annote_list[i][k][0] == doc[j][0]) or (split1[0] == annote_list[i][k][0]) or (split2[0] == annote_list[i][k][0]) or (split3 == annote_list[i][k][0]) :
            
#                 sub_output.append((doc[j][0],doc[j][1],"B-"+annote_list[i][k][1]))
                sub_output.append((doc[j][0],doc[j][1],"B",index_start,index_end))
                #################################################
                sub_error.append(doc[j][0])
                if k < len(annote_list[i])-1:
                    k+=1
                else:
                    break
            #如果是由多个字符组成的，则分别判断
            elif doc[j][0] == split_annote[0] or find_inner1 or find_combin or find_inner2:
                flag=True
#                 sub_output.append((doc[j][0],doc[j][1],"B-"+annote_list[i][k][1]))
                if find_combin:
                    sub_output.append((doc[j][0]+doc[j+1][0],doc[j][1],"B",index_start,index_end))
                    is_jump=True
                else:
                    sub_output.append((doc[j][0],doc[j][1],"B",index_start,index_end))
    
    
                #################################################
                sub_error.append(doc[j][0])
                if flag_index < len(split_annote)-1:
                    flag_index+=1
                

            elif doc[j][0] == split_annote[flag_index] or split2[0] == split_annote[flag_index]:
#                 sub_output.append((doc[j][0],doc[j][1],"I-"+annote_list[i][k][1]))
                #################################################
                sub_error.append(doc[j][0])
                sub_output.append((doc[j][0],doc[j][1],"B",index_start,index_end))

                
                if flag_index<len(split_annote)-1:
                    flag_index+=1
                else:
                    flag_index = 0
                    if k < len(annote_list[i])-1:
                        k+=1
                    else:
                        break


            else:
                flag=False
                flag_index=0
                sub_output.append((doc[j][0],doc[j][1],"O",index_start,index_end))
                
            index_end = index_end + 1
            index_start = index_end
            
        record_error.append(sub_error)
        out_put.append(sub_output)

In [93]:
out_put[0]

[('Intraocular', 'JJ', 'B', 0, 11),
 ('pressure', 'NN', 'O', 12, 20),
 ('in', 'IN', 'O', 21, 23),
 ('genetically', 'RB', 'B', 24, 35),
 ('distinct', 'JJ', 'O', 36, 44),
 ('mice', 'NNS', 'B', 45, 49)]

In [100]:
len(store_list[0])

78

In [101]:
store_list[0]

'Intraocular pressure in genetically distinct mice: an update and strain survey'

In [107]:
error_list = []
for i in range(len(store_list)) :
    for j in out_put[i]:

        if store_list[i][j[3] : j[4]] != j[0]:
            error_list.append((i,j))

In [110]:
store_list[1]

'Little is known about genetic factors affecting intraocular pressure (IOP) in mice and other mammals'

In [112]:
store_list[1][71:74]

'OP)'

In [111]:
out_put[1]

[('Little', 'JJ', 'O', 0, 6),
 ('is', 'VBZ', 'O', 7, 9),
 ('known', 'VBN', 'O', 10, 15),
 ('about', 'IN', 'O', 16, 21),
 ('genetic', 'JJ', 'B', 22, 29),
 ('factors', 'NNS', 'O', 30, 37),
 ('affecting', 'VBG', 'O', 38, 47),
 ('intraocular', 'JJ', 'B', 48, 59),
 ('pressure', 'NN', 'O', 60, 68),
 ('(', '-LRB-', 'O', 69, 70),
 ('IOP', 'NN', 'O', 71, 74),
 (')', '-RRB-', 'O', 75, 76),
 ('in', 'IN', 'O', 77, 79),
 ('mice', 'NNS', 'B', 80, 84),
 ('and', 'CC', 'O', 85, 88),
 ('other', 'JJ', 'O', 89, 94),
 ('mammals', 'NNS', 'B', 95, 102)]

In [108]:
error_list

[(1, ('IOP', 'NN', 'O', 71, 74)),
 (1, (')', '-RRB-', 'O', 75, 76)),
 (1, ('in', 'IN', 'O', 77, 79)),
 (1, ('mice', 'NNS', 'B', 80, 84)),
 (1, ('and', 'CC', 'O', 85, 88)),
 (1, ('other', 'JJ', 'O', 89, 94)),
 (1, ('mammals', 'NNS', 'B', 95, 102)),
 (2, (',', ',', 'O', 90, 91)),
 (2, ('assess', 'VB', 'O', 92, 98)),
 (2, ('the', 'DT', 'O', 99, 102)),
 (2, ('effects', 'NNS', 'O', 103, 110)),
 (2, ('of', 'IN', 'O', 111, 113)),
 (2, ('factors', 'NNS', 'O', 114, 121)),
 (2, ('such', 'JJ', 'O', 122, 126)),
 (2, ('as', 'IN', 'O', 127, 129)),
 (2, ('age', 'NN', 'O', 130, 133)),
 (2, (',', ',', 'O', 134, 135)),
 (2, ('sex', 'NN', 'O', 136, 139)),
 (2, ('and', 'CC', 'O', 140, 143)),
 (2, ('time', 'NN', 'O', 144, 148)),
 (2, ('of', 'IN', 'O', 149, 151)),
 (2, ('day', 'NN', 'O', 152, 155)),
 (2, ('on', 'IN', 'O', 156, 158)),
 (2, ('IOP', 'NN', 'O', 159, 162)),
 (2, ('in', 'IN', 'O', 163, 165)),
 (2, ('specific', 'JJ', 'O', 166, 174)),
 (2, ('strain', 'NN', 'O', 175, 181)),
 (2, ('backgrounds', 'NNS

In [59]:
    right_ann=[]
    for i in annote_list:
        sub_ann=[]
        for j in i:
    #         sub_ann.append(j[0])
            sub_ann += j[0].split()
        right_ann.append(sub_ann)    
        
        
    number_list =[]
    for i in range(len(right_ann)):
        if len(right_ann[i]) != len(record_error[i]):
            number_list.append(i)
    print(number_list)

[225]


In [63]:
right_ann[225]

['Eyes', 'mice', 'acid', 'alcohol', 'formalin', 'fixative']

In [64]:
record_error[225]

['Eyes', 'mice', 'acid-alcohol-formalin']

In [201]:
out_put

[[('Intraocular', 'JJ', 'B', 0, 11),
  ('pressure', 'NN', 'O', 12, 20),
  ('in', 'IN', 'O', 21, 23),
  ('genetically', 'RB', 'B', 24, 35),
  ('distinct', 'JJ', 'O', 36, 44),
  ('mice', 'NNS', 'B', 45, 49)],
 [('Little', 'JJ', 'O', 0, 6),
  ('is', 'VBZ', 'O', 7, 9),
  ('known', 'VBN', 'O', 10, 15),
  ('about', 'IN', 'O', 16, 21),
  ('genetic', 'JJ', 'B', 22, 29),
  ('factors', 'NNS', 'O', 30, 37),
  ('affecting', 'VBG', 'O', 38, 47),
  ('intraocular', 'JJ', 'B', 48, 59),
  ('pressure', 'NN', 'O', 60, 68),
  ('(', '-LRB-', 'O', 69, 70),
  ('IOP', 'NN', 'O', 71, 74),
  (')', '-RRB-', 'O', 75, 76),
  ('in', 'IN', 'O', 77, 79),
  ('mice', 'NNS', 'B', 80, 84),
  ('and', 'CC', 'O', 85, 88),
  ('other', 'JJ', 'O', 89, 94),
  ('mammals', 'NNS', 'B', 95, 102)],
 [('The', 'DT', 'O', 0, 3),
  ('purpose', 'NN', 'O', 4, 11),
  ('of', 'IN', 'O', 12, 14),
  ('this', 'DT', 'O', 15, 19),
  ('study', 'NN', 'O', 20, 25),
  ('was', 'VBD', 'O', 26, 29),
  ('to', 'TO', 'O', 30, 32),
  ('determine', 'VB', 'O'

In [27]:
len(out_put)

235

In [60]:
trainx=[]
trainy=[]
for i in out_put:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        sub_trainx.append(j[0])
        sub_trainy.append(j[2])
    trainx.append(sub_trainx)
    trainy.append(sub_trainy)

In [29]:
import kashgari
kashgari.config.use_cudnn_cell = True
from kashgari.tasks.labeling import BiLSTM_Model

In [61]:
model = BiLSTM_Model()
valid_x, valid_y = trainx[:150], trainy[:150]

model.fit(trainx[:150], trainy[:150], valid_x, valid_y,epochs=50)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 34)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 34, 100)           31200     
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 34, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 34, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 34, 3)             771       
_________________________________________________________________
activation_3 (Activation)    (None, 34, 3)             0         
Total params: 267,491
Trainable params: 267,491
Non-trainable params: 0
_____________________________________________________

Epoch 25/50
3/3 [==============================] - 0s 16ms/step - loss: 0.1480 - acc: 0.9573 - val_loss: 0.1391 - val_acc: 0.9620
Epoch 26/50
3/3 [==============================] - 0s 15ms/step - loss: 0.1441 - acc: 0.9602 - val_loss: 0.1302 - val_acc: 0.9637
Epoch 27/50
3/3 [==============================] - 0s 15ms/step - loss: 0.1410 - acc: 0.9614 - val_loss: 0.1234 - val_acc: 0.9647
Epoch 28/50
3/3 [==============================] - 0s 17ms/step - loss: 0.1325 - acc: 0.9610 - val_loss: 0.1313 - val_acc: 0.9600
Epoch 29/50
3/3 [==============================] - 0s 15ms/step - loss: 0.1338 - acc: 0.9616 - val_loss: 0.1275 - val_acc: 0.9631
Epoch 30/50
3/3 [==============================] - 0s 15ms/step - loss: 0.1242 - acc: 0.9629 - val_loss: 0.1168 - val_acc: 0.9633
Epoch 31/50
3/3 [==============================] - 0s 15ms/step - loss: 0.1283 - acc: 0.9641 - val_loss: 0.1111 - val_acc: 0.9676
Epoch 32/50
3/3 [==============================] - 0s 16ms/step - loss: 0.1192 - acc: 0.96

In [62]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(trainx[150:], trainy[150:])

           precision    recall  f1-score   support

        B     0.9920    0.6108    0.7561       203

micro avg     0.8435    0.6108    0.7086       203
macro avg     0.9920    0.6108    0.7561       203



'           precision    recall  f1-score   support\n\n        B     0.9920    0.6108    0.7561       203\n\nmicro avg     0.8435    0.6108    0.7086       203\nmacro avg     0.9920    0.6108    0.7561       203\n'